In [ ]:
# set problem parameters
M = 100
var_1 = 0.1
var = 3
a = 1
sample_mean = 0  # assumed to be exactly =0

eps = [0.75, 0.25, 0.1, 0.025]  # tolerances

def abc_mcmc(N_max, N_eff, M, eps, nu_squared, var, var_1, a):
    theta = np.zeros(N_max+1)
    pi_rv = st.norm(loc=0, scale=nu_squared)  # prior ?
    acceptance_count = 0
    rejection_count = 0
    entered = 0
    i = 0
    while (acceptance_count < N_eff) & (i < N_max):
        # sample candidate parameters from a proposal transition density q(theta_i, )
        # random walk proposal q(theta, ) = N(theta,nu_squared)
        q_i = st.norm(loc=theta[i], scale=nu_squared)
        theta_star = q_i.rvs()
    
        # generate data from the underlying model given theta_star
        # generate_data function: TODO
        N1_rv = st.norm(loc=theta_star, scale=var_1)
        N2_rv = st.norm(loc=theta_star + a, scale=var_1)
        D = np.zeros(M)
        for j in range(M):
            if st.uniform.rvs(0,1) < 0.5:
                D[j] = N1_rv.rvs()
                # n1_count = n1_count + 1
            else:
                D[j] = N2_rv.rvs()
                # n2_count = n1_count + 1
    
        if discrepancy_metric(D) < eps:
            q_star = st.norm(loc=theta_star, scale=nu_squared)
            comp = pi_rv.pdf(theta_star)*q_star.pdf(theta[i])/( pi_rv.pdf(theta[i])*q_i.pdf(theta_star) )

            entered = entered + 1
            if st.uniform.rvs() < comp:
                theta[i+1] = theta_star
                acceptance_count = acceptance_count + 1 
            else:
                theta[i+1] = theta[i]
                rejection_count = rejection_count + 1
        else:
            theta[i+1] = theta[i]
            rejection_count = rejection_count + 1
        i = i + 1
    
    print(acceptance_count, entered, rejection_count, i) ## DEBUG
    if acceptance_count == 500:
        print('effective sample size achieved')
    print('acceptance rate:', acceptance_count/(acceptance_count+rejection_count))
    return theta

In [ ]:
N_max = 5000  # max number of accepted samples ### TEMP
N_eff = 500  # effective size
nu_squared = var_1  ### TEMP

x_values = np.linspace(-1.25, 0.5, 500)
f = true_mixture_distribution(x_values, M, a, var, var_1, sample_mean=0)

fig, axs = plt.subplots(2, figsize=(10,12))
for e in eps:
    theta = abc_mcmc(N_max, N_eff, M, e, nu_squared, var, var_1, a)
    label = 'eps = %s'%(e)
    axs[0].hist(theta, bins=30, alpha=0.7, label=label)
    axs[1].plot(theta, label=label)
axs[0].plot(x_values, f, label='True Mixture Distribution')
axs[0].legend()
axs[1].legend()
plt.show()